In [22]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd

In [7]:
movie_summaries = pd.read_csv("/Users/nhatanhdao/Documents/EPFL/MA1/ada/Project/MovieSummaries/plot_summaries.txt", header=None, sep="\t")
movie_summaries.columns = ["Movie ID", "Plot Summary"]
movie_summaries.head()

,Movie ID,Plot Summary
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...


In [8]:
tv_tropes = pd.read_csv("/Users/nhatanhdao/Documents/EPFL/MA1/ada/Project/MovieSummaries/tvtropes.clusters.txt", header=None, sep="\t")
tv_tropes.columns = ["Trope", "Characteristics"]
tv_tropes.head()

,Trope,Characteristics
0,absent_minded_professor,"{""char"": ""Professor Philip Brainard"", ""movie"":..."
1,absent_minded_professor,"{""char"": ""Professor Keenbean"", ""movie"": ""Richi..."
2,absent_minded_professor,"{""char"": ""Dr. Reinhardt Lane"", ""movie"": ""The S..."
3,absent_minded_professor,"{""char"": ""Dr. Harold Medford"", ""movie"": ""Them!..."
4,absent_minded_professor,"{""char"": ""Daniel Jackson"", ""movie"": ""Stargate""..."


In [9]:
# Initialize the model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for movie plots
plot_embeddings = model.encode(movie_summaries['Plot Summary'].tolist())

# Generate embeddings for tropes
trope_embeddings = model.encode(tv_tropes['Trope'].tolist())  

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
# Calculate cosine similarity between each plot and trope
similarity_scores = util.cos_sim(plot_embeddings, trope_embeddings)

# Threshold for similarity
threshold = 0.5

# Match tropes to plots
matches = []
for plot_idx, scores in enumerate(similarity_scores):
    matched_tropes = []
    for trope_idx, score in enumerate(scores):
        if score > threshold:
            matched_tropes.append(tv_tropes['Trope'].iloc[trope_idx])
    matches.append({'plot': movie_summaries['Plot Summary'].iloc[plot_idx], 'matched_tropes': matched_tropes})

In [26]:
# Convert results to a DataFrame
matches_df = pd.DataFrame(matches)

# Remove rows where there are no matches
matches_df = matches_df[matches_df['matched_tropes'].apply(len) > 0]

# Save the final dataframe to a CSV file
matches_df.to_csv("movies_with_tropes.csv", index=False)

matches_df.head()


,plot,matched_tropes
113,A thief named Shyam who has made stealing his...,"[gentleman_thief, gentleman_thief, gentleman_t..."
1513,The outlaw Speedy Gonzales rides into town in ...,"[retired_outlaw, retired_outlaw, retired_outla..."
1993,A villainous rival sends in four double agents...,"[revenge, revenge, revenge, revenge, revenge, ..."
3988,Slacker is a uniquely structured and seemingly...,"[slacker, slacker, slacker, slacker, slacker, ..."
8720,Notorious gunslinger George Kelby Jr. and his...,"[fastest_gun_in_the_west, fastest_gun_in_the_w..."
